# Fine-Tuning Code Llama for SQL Query Generation

This notebook demonstrates how to fine-tune the Code Llama model to become an expert SQL developer using the Hugging Face library. I will use the `b-mc2/sql-create-context` dataset, which contains text queries and their corresponding SQL queries. The approach involves using LoRA (Low-Rank Adaptation) to efficiently train the model by freezing its weights and only training an adapter. I will also quantize the model to int8 to optimize performance and use Weights & Biases (W&B) for experiment tracking. Let's get started!

## Step 1: Set up the environment

In this step, I will set up the Kaggle environment by installing the necessary libraries. This includes the Hugging Face Transformers and Datasets libraries for model handling and dataset management, `bitsandbytes` for model quantization, and Weights & Biases (W&B) for tracking our experiments.

In [1]:
   !pip install transformers datasets accelerate
   !pip install bitsandbytes
   !pip install wandb
   !pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.1 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-p2za2t_z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-p2za2t_z
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536616888d51b453ed354a6f1e243fecb02ea08
  Resolved https://github.com/huggingface/peft.git to commit e536616888d51b453ed354a6f1e243fecb02ea08
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=41638 sha256=580eee682a5502d10b4eb80f43411a10eabdc10044493ae12

In [2]:
import os
from kaggle_secrets import UserSecretsClient

import wandb
import matplotlib.pyplot as plt

import torch
from datasets import load_dataset
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq)
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
    PeftModel
) 

Next, I will import the necessary libraries and initialize W&B for tracking our experiments. This will help us monitor the training process and evaluate the model's performance.

In [ ]:
user_secrets = UserSecretsClient() 

personal_key_for_api = user_secrets.get_secret("wandb-key")

! wandb login $personal_key_for_api

## Step 2: Exploratory Data Analysis (EDA)

Before training the model, it's important to understand the dataset. I will load the `b-mc2/sql-create-context` dataset and perform exploratory data analysis (EDA) to gain insights into the data. This includes examining the structure of the dataset, analyzing the distribution of text and SQL query lengths, and visualizing these distributions.

In [ ]:
dataset = load_dataset("b-mc2/sql-create-context",split="train")
train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]


In [ ]:
train_dataset

In [ ]:
eval_dataset

In [ ]:
print(train_dataset[0])

In [ ]:
feature = "answer" # context, answer

lengths = [len(sample[feature]) for sample in train_dataset]
plt.hist(lengths, bins=50)
plt.title(f'Distribution of {feature} lengths')
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.show()

## Step 3: Model preparation

In this step, I will prepare the Code Llama model for fine-tuning. I will load the pre-trained model and tokenizer, quantize the model to int8 using `bitsandbytes`, and freeze the model weights to train only the adapter. I will implement a LoRA adapter to efficiently adapt the model for SQL query generation.

In [ ]:
model_name = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Testing the original model

Before fine-tunining the model, it is convenient to check the base model's ability to perform the task at hand.

In [ ]:
eval_prompt = """You are an SQL expert. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

This not the correct answer

In [ ]:
model.train()
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config)
model.print_trainable_parameters()

## Step 4: Training

With the model prepared, I will now proceed to train it on the dataset. I will tokenize the dataset, set up training arguments, and use the Hugging Face Trainer to fine-tune the model.

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [ ]:
def tokenize_function(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
{data_point["answer"]}
"""
    return tokenize_function(full_prompt)

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

In [ ]:
output_dir = "sql-code-llama"

training_args = TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=1,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=20,
        save_steps=20,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="wandb", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )
model.config.use_cache = False 


In [ ]:

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [ ]:
trainer.train()

trainer.model.save_pretrained("trained-model")
tokenizer.save_pretrained("tokenizer")

## Step 5: Evaluation

I will now load the trained model for inference

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [4]:
output_dir = "/kaggle/input/code-llama-sql/transformers/default/1/sql-code-llama/checkpoint-400"
model = PeftModel.from_pretrained(model, output_dir)

/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [5]:
#for i in model.named_parameters():
#    print(f"{i[0]} -> {i[1].device}")

In [11]:
def postprocess_model_output(model_output: str) -> str:
    """
    Extracts the SQL query from the model's output by finding the text after the '### Response:' marker.

    Parameters:
    - model_output (str): The full output string from the model.

    Returns:
    - str: The extracted SQL query.
    """
    # Define the marker that indicates the start of the response
    response_marker = "### Response:"

    # Find the position of the response marker
    response_start = model_output.find(response_marker)

    # If the marker is found, extract the text following it
    if response_start != -1:
      # Extract the response text, stripping any leading/trailing whitespace
        response_text = model_output[response_start + len(response_marker):].strip()
        return response_text
    else:
      # If the marker is not found, return an empty string or handle as needed
        return ""

In [10]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
model_input = tokenizer(eval_prompt, return_tensors="pt")

model.eval()
with torch.no_grad():
    print(postprocess_model_output(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


SELECT class FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = "hyannis, nebraska"


The model now outputs the correct answer.

## Conclusion

In this notebook, I successfully fine-tuned the Code Llama model to generate SQL queries using the `b-mc2/sql-create-context` dataset. I utilized a LoRA approach to efficiently train the model by freezing its weights and only training an adapter. The model was quantized to int8 for optimized performance, and I tracked our experiments using Weights & Biases. This setup provides a robust framework for transforming Code Llama into a proficient SQL developer model.